In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
import math
from bayes_opt import BayesianOptimization
import random
import sklearn
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [2]:


# Step 1: Load your dataset. Make sure you have a CSV file with relevant features and effort (target) column.
# Replace 'your_dataset.csv' with your actual dataset file name and adjust the column names accordingly.
maxwell=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\maxwell_dataset.csv",header=None)
columns_maxwell=['Syear','App','Har','Dba','Ifc','Source','Telonuse','Nlan','T01','T02','T03','T04','T05','T06','T07','T08','T09','T10','T11','T12','T13','T14','T15','Duration','Size','Time','Effort']
maxwell.set_axis(columns_maxwell,axis='columns',inplace=True)
maxwell.set_axis(range(1,63),axis=0 ,inplace=True)
maxwell.rename_axis("Features", axis=1,inplace=True)
maxwell.rename_axis("Projects", axis=0,inplace=True)
dataset = maxwell

# Step 2: Prepare the data.
X = dataset.drop(columns=['Effort']).values  # Features
y = dataset['Effort'].values  # Target (effort)


seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.function(reduce_retracing=True)


# Step 3: Feature Selection using Harmony Search
def objective_function(subset):
    selected_features = [feature for feature, is_selected in zip(range(X.shape[1]), subset) if is_selected]
    if len(selected_features) == 0:
        return float('-inf')  # Penalize subsets with no selected features
    
    
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    cross_val_rmse = []
    
    X_selected = X[:, selected_features]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    error_list=[]
    
    for train_index, val_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[val_index]
        y_train, y_test = y[train_index], y[val_index]
    
        # Step 3: Build the ANN model.
        model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(units=16, activation='relu'),
        tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
    ])

        # Step 4: Compile the model.
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Step 5: Train the model.
        model.fit(X_train, y_train, epochs=5, batch_size=8, verbose=0)

        # Step 6: Evaluate the model.
        y_pred = model.predict(X_test)
    
        error = np.mean(np.abs(y_pred - y_test))
        error_list.append(error)
        
    return 1 / (1 + np.mean(error_list))  # Fitness is the inverse of the error (higher is better)
#====================================================================================================================
# Harmony Search Parameters
num_features = X.shape[1]
hms = 20  # Harmony memory size
iterations = 100  # Number of iterations

# Initialize harmony memory
harmony_memory = np.random.randint(0, 2, size=(hms, num_features))

# Harmony Search Algorithm
for _ in range(iterations):
    rnd_choice=np.random.choice(hms)
    new_harmony = np.copy(harmony_memory[rnd_choice])
    for i in range(num_features):
        if np.random.rand() < 0.5:  # Adjust this probability based on your problem
            new_harmony[i] = 1 - new_harmony[i]
    current_obj = objective_function(harmony_memory[rnd_choice])
    new_obj = objective_function(new_harmony)
    if new_obj < current_obj:
        harmony_memory[rnd_choice] = new_harmony

# Get selected features based on final harmony memory
selected_features = harmony_memory[np.argmax([objective_function(i) for i in harmony_memory])]
print(selected_features)
#======================================================================================================================

#if len(selected_features) == 0:
#   return float('-inf')  # Penalize subsets with no selected features
    
    
# Define the ANN model to be optimized.
def ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=int(neurons_input), activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(int(num_hidden_layers)):
        model.add(tf.keras.layers.Dense(units=int(neurons_hidden), activation='relu'))

    model.add(tf.keras.layers.Dense(units=1))  # Output layer with a single unit for regression.

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

# Define the search space for Bayesian optimization.
pbounds = {
        'neurons_input': (10, 50),
        'neurons_hidden': (10, 50),
        'num_hidden_layers': (1, 5),
        'learning_rate': (1e-5, 1e-2),
        'batch_size': (8,32 ),
        'epochs': (5, 20)
}

# Define the function to optimize (minimize RMSE).
def optimize_effort_estimation(neurons_input, neurons_hidden, num_hidden_layers, learning_rate, batch_size, epochs):
    model = ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate)
    
    model.fit(X_train, y_train, batch_size=int(batch_size), epochs=int(epochs), verbose=0)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Minimize the negative RMSE for Bayesian optimization.

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
X_selected = X[:, selected_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
mean_MAE=[]
mean_MMRE=[]
mean_RMSE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    # Perform Bayesian optimization.
    optimizer = BayesianOptimization(
        f=optimize_effort_estimation,
        pbounds=pbounds,
        random_state=42,
        verbose=2,
    )

    optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.

    # Print the best hyperparameters found.
    best_params = optimizer.max['params']
        
    #build the model
    model = ann_model(best_params['neurons_input'],best_params['neurons_hidden'],best_params['num_hidden_layers'],best_params['learning_rate'])
    # Train the model.
    model.fit(X_train, y_train, epochs=int(best_params['epochs']), batch_size=int(best_params['batch_size']), verbose=0)

    # Step 6: Evaluate the model.
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)


    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
                
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")               
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")            
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")

1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 41ms/step


1/1 [==============================] - 0s 39ms/step


...

1/1 [==============================] - 0s 34ms/step
| 31        | -1.852e+0 | 8.0       | 20.0      | 0.01      | 38.88     | 10.69     | 1.0       |
1/1 [==============================] - 0s 33ms/step
| 32        | -1.898e+0 | 8.0       | 15.41     | 0.01      | 18.72     | 39.59     | 1.079     |
1/1 [==============================] - 0s 48ms/step
| 33        | -1.701e+0 | 14.38     | 20.0      | 0.008175  | 14.17     | 37.11     | 5.0       |
1/1 [==============================] - 0s 48ms/step
| 34        | -1.705e+0 | 8.819     | 20.0      | 0.01      | 11.04     | 37.88     | 5.0       |
1/1 [==============================] - 0s 48ms/step
| 35        | -1.709e+0 | 13.55     | 20.0      | 0.01      | 11.18     | 43.62     | 5.0       |
1/1 [==============================] - 0s 47ms/step
| 36        | -1.795e+0 | 13.33     | 14.37     | 0.01      | 10.0      | 37.65     | 5.0       |
1/1 [==============================] - 0s 37ms/step
| 37        | -1.777e+0 | 10.91     | 18.8      

1/1 [==============================] - 0s 47ms/step
| 28        | -1.009e+0 | 8.0       | 20.0      | 0.01      | 34.99     | 18.84     | 5.0       |
1/1 [==============================] - 0s 47ms/step
| 29        | -1.011e+0 | 8.0       | 20.0      | 0.01      | 31.79     | 24.24     | 5.0       |
1/1 [==============================] - 0s 46ms/step
| 30        | -9.992e+0 | 8.0       | 20.0      | 0.01      | 33.17     | 10.07     | 5.0       |
1/1 [==============================] - 0s 49ms/step
| 31        | -1.006e+0 | 8.0       | 20.0      | 0.01      | 44.79     | 14.32     | 5.0       |
1/1 [==============================] - 0s 47ms/step
| 32        | -1.016e+0 | 8.0       | 20.0      | 0.01      | 21.3      | 42.14     | 5.0       |
1/1 [==============================] - 0s 47ms/step
| 33        | -1.02e+04 | 8.0       | 12.31     | 0.01      | 28.23     | 46.31     | 5.0       |
1/1 [==============================] - 0s 47ms/step
| 34        | -9.984e+0 | 11.44     | 20.0      

1/1 [==============================] - 0s 42ms/step
| 25        | -4.756e+0 | 27.84     | 6.848     | 0.002967  | 16.16     | 22.18     | 3.74      |
1/1 [==============================] - 0s 37ms/step
| 26        | -4.753e+0 | 17.76     | 19.12     | 0.001242  | 32.64     | 16.5      | 2.477     |
1/1 [==============================] - 0s 38ms/step
| 27        | -4.629e+0 | 22.15     | 6.854     | 0.008468  | 43.81     | 48.25     | 2.331     |
1/1 [==============================] - 0s 45ms/step
| 28        | -8.427e+0 | 21.43     | 8.301     | 0.008637  | 40.59     | 48.73     | 4.367     |
1/1 [==============================] - 0s 33ms/step
| 29        | -4.755e+0 | 29.95     | 6.991     | 0.004999  | 41.0      | 30.14     | 1.329     |
1/1 [==============================] - 0s 45ms/step
| 30        | -4.748e+0 | 23.13     | 8.055     | 0.003133  | 17.94     | 16.38     | 4.888     |
1/1 [==============================] - 0s 32ms/step
| 31        | -4.757e+0 | 27.88     | 15.54     

1/1 [==============================] - 0s 34ms/step
| 22        | -6.962e+0 | 10.98     | 19.71     | 0.002799  | 46.32     | 31.03     | 1.66      |
1/1 [==============================] - 0s 51ms/step
| 23        | -4.883e+0 | 8.0       | 14.38     | 0.01      | 36.85     | 27.11     | 5.0       |
1/1 [==============================] - 0s 34ms/step
| 24        | -7.107e+0 | 8.94      | 5.121     | 0.005121  | 39.32     | 28.8      | 1.651     |
1/1 [==============================] - 0s 45ms/step
| 25        | -4.882e+0 | 9.844     | 19.11     | 0.006471  | 34.41     | 30.92     | 4.969     |
1/1 [==============================] - 0s 48ms/step
| 26        | -7.131e+0 | 8.0       | 5.0       | 1e-05     | 10.0      | 50.0      | 5.0       |
1/1 [==============================] - 0s 44ms/step
| 27        | -5.32e+03 | 9.116     | 15.01     | 0.001843  | 38.18     | 39.89     | 4.42      |
1/1 [==============================] - 0s 47ms/step
| 28        | -7.131e+0 | 32.0      | 20.0      

1/1 [==============================] - 0s 40ms/step
| 19        | -1.413e+0 | 25.33     | 5.43      | 0.007739  | 36.66     | 37.6      | 3.145     |
1/1 [==============================] - 0s 33ms/step
| 20        | -1.414e+0 | 29.01     | 10.02     | 0.003366  | 47.63     | 31.9      | 1.897     |
1/1 [==============================] - 0s 39ms/step
| 21        | -1.376e+0 | 18.53     | 19.72     | 0.002655  | 30.99     | 21.28     | 3.285     |
1/1 [==============================] - 0s 37ms/step
| 22        | -1.225e+0 | 10.85     | 10.28     | 0.007454  | 37.26     | 32.57     | 2.429     |
1/1 [==============================] - 0s 34ms/step
| 23        | -1.413e+0 | 16.34     | 15.16     | 0.004878  | 13.67     | 10.61     | 1.883     |
1/1 [==============================] - 0s 40ms/step
| 24        | -1.318e+0 | 23.59     | 18.05     | 0.005597  | 23.23     | 48.02     | 3.229     |
1/1 [==============================] - 0s 39ms/step
| 25        | -1.414e+0 | 20.57     | 7.095     